In [ ]:
import pandas as pd

train_df = pd.read_csv('./train_preprocessing.csv')
test_df = pd.read_csv('./test_preprocessing.csv')

## train_df/test_df : 개발자가 전처리한 데이터셋을 csv 형태로 저장한 파일

In [ ]:
from sklearn.model_selection import train_test_split

# target, feature 분리
target = train_df['Survived']
train_df.drop(['Survived'], axis = 1, inplace = True)

X_train, X_test, y_train, y_test=train_test_split(train_df, target, test_size=0.2, random_state=42)

In [ ]:
Pid_df = test_df['PassengerId']

In [ ]:
#h2o
import h2o
from h2o.automl import H2OAutoML

train_df = pd.read_csv('./train_df.csv')

h2o.init()

train_df_h2o = h2o.H2OFrame(train_df)
test_df_h2o = h2o.H2OFrame(test_df)

predictors = train_df_h2o.columns
response = "Survived"
predictors.remove(response)

train_df_h2o[response] = train_df_h2o[response].asfactor()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_392"; OpenJDK Runtime Environment (build 1.8.0_392-8u392-ga-1~22.04-b08); OpenJDK 64-Bit Server VM (build 25.392-b08, mixed mode)
  Starting server from /home/dummy/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpza1tkirs
  JVM stdout: /tmp/tmpza1tkirs/h2o_dummy_started_from_python.out
  JVM stderr: /tmp/tmpza1tkirs/h2o_dummy_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,H2O_from_python_dummy_ykbkrs
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.456 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
import math
import time

train, valid = train_df_h2o.split_frame(ratios = [.8])

# predictors.remove(response)


aml = H2OAutoML(max_models=20, seed=1)

start = time.time()
aml.train(x=predictors, y=response, training_frame=train_df_h2o)
end = time.time()

print("time :", end - start)

# 126 ~ 130초

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
time : 131.13329935073853


In [ ]:
best = h2o.automl.get_leaderboard(aml)
best

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_2_20240212_183240,0.872855,0.410074,0.847768,0.170981,0.353075,0.124662
DeepLearning_grid_2_AutoML_2_20240212_183240_model_1,0.871973,0.482129,0.844142,0.177356,0.358971,0.12886
GBM_2_AutoML_2_20240212_183240,0.870943,0.413344,0.847727,0.181454,0.35668,0.127221
XRT_1_AutoML_2_20240212_183240,0.870671,0.424807,0.838385,0.185649,0.364605,0.132937
XGBoost_grid_1_AutoML_2_20240212_183240_model_3,0.870498,0.412753,0.841751,0.167529,0.353988,0.125307
XGBoost_2_AutoML_2_20240212_183240,0.870253,0.419198,0.839598,0.186775,0.36094,0.130277
GBM_3_AutoML_2_20240212_183240,0.870213,0.413874,0.844518,0.168656,0.355402,0.126311
GBM_5_AutoML_2_20240212_183240,0.869947,0.416671,0.842316,0.182173,0.359718,0.129397
GBM_4_AutoML_2_20240212_183240,0.868959,0.417944,0.846253,0.179249,0.358772,0.128717
StackedEnsemble_AllModels_1_AutoML_2_20240212_183240,0.868765,0.410453,0.845224,0.174072,0.354386,0.12559


In [ ]:
best_model = h2o.get_model('StackedEnsemble_BestOfFamily_1_AutoML_2_20240212_183240')
type(best_model)
best_model

key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [ ]:
print(best_model.base_models)
best_model.actual_params

['DeepLearning_grid_2_AutoML_2_20240212_183240_model_1', 'GBM_2_AutoML_2_20240212_183240', 'XRT_1_AutoML_2_20240212_183240', 'XGBoost_grid_1_AutoML_2_20240212_183240_model_3', 'DRF_1_AutoML_2_20240212_183240', 'GLM_1_AutoML_2_20240212_183240']


In [ ]:
from sklearn.metrics import accuracy_score

X_test_h2o = h2o.H2OFrame(X_test)
pred = best_model.predict(X_test_h2o)

y_pred = pred['predict'].as_data_frame()

accuracy_score(y_test, y_pred)

# test accuracy : 0.8882

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/home/dummy/.local/lib/python3.10/site-packages/h2o/frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


0.888268156424581

In [ ]:
y_pred = leader.predict(test_df_h2o)
y_pred = y_pred['predict'].as_data_frame()
pred = y_pred['predict']

submission = pd.DataFrame({
        "PassengerId": Pid_df,
        "Survived": pred
    })
submission.to_csv('submission_h2o.csv', index=False)
# kaggle score : 0.73927

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/home/dummy/.local/lib/python3.10/site-packages/h2o/frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [ ]:
# auto Keras
import pandas as pd
import tensorflow as tf

import autokeras as ak

'''
train_df = pd.read_csv('./train_df.csv')
test_df = pd.read_csv('./test_df.csv')
'''

#train_df = './train_df.csv'
#test_df = './test_df.csv'


%time
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=10
)  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(X_train, y_train,validation_data=(X_test, y_test),epochs=10)

# 01m 09s
# test accuracy = 0.7933

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.5921787619590759

Best val_accuracy So Far: 0.8156424760818481
Total elapsed time: 00h 00m 46s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Epoch 1/10
23/23 [==============================] - 1s 13ms/step - loss: 0.5415 - accuracy: 0.7542 - val_loss: 0.4911 - val_accuracy: 0.7765
Epoch 2/10
23/23 [==============================] - 0s 9ms/step - loss: 0.4591 - accuracy: 0.8118 - val_loss: 0.4840 - val_accuracy: 0.7821
Epoch 3/10
23/23 [==============================] - 0s 8ms/step - loss: 0.4308 - accuracy: 0.8160 - val_loss: 0.4781 - val_accuracy: 0.7877
Epoch 4/10
23/23 [==============================] - 0s 8ms/step - loss: 0.4247 - accuracy: 0.8160 - val_loss: 0.4686 - val_accuracy: 0.7989
Epoch 5/10
23/23 [==============================] - 0s 8ms/step - loss: 0.4148 - accuracy: 0.8244 - val_loss: 0.4655 - val_accuracy: 0.7989
Epoch 6/10
23/23 [==============================] - 0s 8ms/step - loss: 0.4021 - accuracy: 0.8287 - val_loss: 0.4655 - val_accuracy: 0.8045
Epoch 7/10
23/23 [==============================] - 0s 8ms/step - loss: 0.4048 - accuracy: 0.8287 - val_loss: 0.4660 - val_accuracy: 0.8045
Epoch 8/10
23/23 [=

INFO:tensorflow:Assets written to: ./structured_data_classifier/best_model/assets


In [ ]:
# Predict with the best model.
#predicted_y = clf.predict(test_df)
# Evaluate the best model with testing data.
y_pred = clf.evaluate(X_test, y_test)

model = clf.export_model()
model.summary()

6/6 [==============================] - 0s 7ms/step - loss: 0.4612 - accuracy: 0.7933
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 multi_category_encoding (M  (None, 9)                 0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 9)                 19        
 on)                                                             
                                                                 
 dense (Dense)               (None, 16)                160       
                                                                 
 re_lu (ReLU)                (None, 16)                0         
                                          

In [ ]:
y_pred = model.predict(test_df)

14/14 [==============================] - 0s 3ms/step


In [ ]:
y_pred = model.predict(test_df)
y_pred = y_pred.flatten().astype(int)

submission = pd.DataFrame({
        "PassengerId": Pid_df,
        "Survived": y_pred
    })
submission.to_csv('submission_autokeras.csv', index=False)

# kaggle score : 0.75598

14/14 [==============================] - 0s 4ms/step


In [ ]:
# Auto Gluon 사용을 위해 csv file 재연결
train_df = pd.read_csv('./train_preprocessing.csv')
test_df = pd.read_csv('./test_preprocessing.csv')

X_train, X_test, y_train, y_test=train_test_split(train_df, target, test_size=0.2, random_state=42)

In [ ]:
#Auto gluon
from autogluon.tabular import TabularDataset, TabularPredictor

label_str = "Survived"
save_path = 'autogluon_titanic'

start = time.time()
predictor = TabularPredictor(label = label_str, path = save_path).fit(X_train)
end = time.time()

print("time :", end - start)

# 5.64s
# 6.9426초

/home/dummy/.local/lib/python3.10/site-packages/autogluon/core/utils/utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Beginning AutoGluon training ...
AutoGluon will save models to "autogluon_titanic/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct 5 21:02:42 UTC 2023
Disk Space Avail:   960.99 GB / 1081.10 GB (88.9%)
Train Data Rows:    712
Train Data Columns: 9
Label Column: Survived
Preprocessing data ...
/home/dummy/.local/lib/python3.10/site-packages/autogluon/core/utils/utils.py:549: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, 

time : 7.4353086948394775


In [ ]:
df = predictor.leaderboard(X_test)
df

                  model  score_test  score_val  pred_time_test  pred_time_val  fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0              CatBoost    0.821229   0.818182        0.001358       0.001679  0.302098                 0.001358                0.001679           0.302098            1       True          7
1         LightGBMLarge    0.810056   0.769231        0.002126       0.000958  0.733707                 0.002126                0.000958           0.733707            1       True         13
2      RandomForestGini    0.810056   0.783217        0.053860       0.047351  0.477553                 0.053860                0.047351           0.477553            1       True          5
3               XGBoost    0.804469   0.783217        0.009599       0.002566  0.094655                 0.009599                0.002566           0.094655            1       True         11
4      RandomForestEntr    0.804469   0.77622

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,0.821229,0.818182,0.001358,0.001679,0.302098,0.001358,0.001679,0.302098,1,True,7
1,LightGBMLarge,0.810056,0.769231,0.002126,0.000958,0.733707,0.002126,0.000958,0.733707,1,True,13
2,RandomForestGini,0.810056,0.783217,0.053860,0.047351,0.477553,0.053860,0.047351,0.477553,1,True,5
3,XGBoost,0.804469,0.783217,0.009599,0.002566,0.094655,0.009599,0.002566,0.094655,1,True,11
4,RandomForestEntr,0.804469,0.776224,0.051515,0.047019,0.443208,0.051515,0.047019,0.443208,1,True,6
5,LightGBM,0.798883,0.790210,0.000819,0.001046,0.590608,0.000819,0.001046,0.590608,1,True,4
6,LightGBMXT,0.798883,0.818182,0.001463,0.001338,0.829560,0.001463,0.001338,0.829560,1,True,3
7,ExtraTreesEntr,0.798883,0.790210,0.051858,0.046965,0.448641,0.051858,0.046965,0.448641,1,True,9
8,ExtraTreesGini,0.793296,0.790210,0.051672,0.049186,0.462980,0.051672,0.049186,0.462980,1,True,8
9,NeuralNetTorch,0.787709,0.804196,0.006914,0.004726,0.812658,0.006914,0.004726,0.812658,1,True,12


In [ ]:
best_atg = predictor.get_model_full_dict
predictor.info()

{'path': 'autogluon_titanic/',
 'label': 'Survived',
 'random_state': 0,
 'version': '0.8.2',
 'features': ['PassengerId',
  'Pclass',
  'Sex',
  'Age',
  'Fare',
  'Embarked',
  'Title',
  'FamilySize',
  'IsAlone'],
 'feature_metadata_in': <autogluon.common.features.feature_metadata.FeatureMetadata at 0x7f791c39a1a0>,
 'time_fit_preprocessing': 0.032318830490112305,
 'time_fit_training': 7.390127420425415,
 'time_fit_total': 7.422446250915527,
 'time_limit': None,
 'time_train_start': 1707732144.9070137,
 'num_rows_train': 569,
 'num_cols_train': 9,
 'num_rows_val': 143,
 'num_classes': 2,
 'problem_type': 'binary',
 'eval_metric': 'accuracy',
 'best_model': 'WeightedEnsemble_L2',
 'best_model_score_val': 0.8251748251748252,
 'best_model_stack_level': 2,
 'num_models_trained': 14,
 'num_bag_folds': 0,
 'max_stack_level': 2,
 'max_core_stack_level': 1,
 'model_info': {'KNeighborsUnif': {'name': 'KNeighborsUnif',
   'model_type': 'KNNModel',
   'problem_type': 'binary',
   'eval_metric

In [ ]:
from sklearn.metrics import accuracy_score

predictor = TabularPredictor.load(save_path)
Y_pred_atg = predictor.predict(X_test)
atg_acc = round(accuracy_score(y_test,Y_pred_atg), 4)
atg_acc

b = predictor.get_model_best
# test accuracy : 0.7877

In [ ]:
y_pred = predictor.predict(test_df)
submission = pd.DataFrame({
        "PassengerId": Pid_df,
        "Survived": y_pred
    })
submission.to_csv('submission_autogluon.csv', index=False)
#kaggle score : 0.77751

# Auto Gluon에서 best로 뽑은 모델의 하이퍼 파라미터 (Extra Trees model )
 'hyperparameters': {'n_estimators': 300,
    'max_leaf_nodes': 15000,
    'n_jobs': -1,
    'random_state': 0,
    'bootstrap': True,
    'criterion': 'gini'}

In [ ]:
predictor.leaderboard(X_test)

                  model  score_test  score_val  pred_time_test  pred_time_val  fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0              CatBoost    0.821229   0.818182        0.001459       0.001136  0.177462                 0.001459                0.001136           0.177462            1       True          7
1         LightGBMLarge    0.810056   0.769231        0.001071       0.001556  1.759132                 0.001071                0.001556           1.759132            1       True         13
2      RandomForestGini    0.810056   0.783217        0.053163       0.048133  0.459856                 0.053163                0.048133           0.459856            1       True          5
3               XGBoost    0.804469   0.783217        0.011581       0.002438  0.076096                 0.011581                0.002438           0.076096            1       True         11
4      RandomForestEntr    0.804469   0.77622

AttributeError: 'DataFrame' object has no attribute 'get_info'